In [3]:
import pandas as pd
from bs4 import BeautifulSoup

In [99]:
# get the response in the form of html
ww2_url= "https://en.wikipedia.org/wiki/World_War_II_casualties"
ww2_df = pd.read_html(ww2_url)[0]

ww1_url= "https://en.wikipedia.org/wiki/World_War_I_casualties"
ww1_df = pd.read_html(ww1_url)[0]
ww1_df.columns = ww1_df.columns.droplevel(1)  #remove multi-index layer

#Drop non-duplicated data
ww1_df = ww1_df.drop(['Combat deaths and missing in action (included in total military deaths)'], axis=1) 
ww2_df = ww2_df.drop(columns = ww2_df.columns[7])

#Drop index rows
ww1_df = ww1_df[ww1_df['Nation'] != 'Central Powers']
ww1_df = ww1_df[ww1_df['Nation'] != 'TotalAllied Powers']
ww1_df = ww1_df[ww1_df['Nation'] != 'TotalCentral Powers']
ww1_df = ww1_df[ww1_df['Nation'] != 'Neutral nations']
ww1_df = ww1_df[ww1_df['Nation'] != 'Grand total']
ww1_df = ww1_df[ww1_df['Nation'] != 'Approx. totals']


#Set column names
names = ["Nation", "Population", "Total military deaths from all causes", "Civilian deaths due to direct military action", 
"Civilian deaths due to indirect military action","Total deaths", 'Deaths as percent of population', 'Military wounded']
ww1_df.columns = names
ww2_df.columns = names



# Cleaning WW1 Data

# Removes all things in brackets and converts to standard numbers
ww1_df['Population'] = ww1_df['Population'].str.replace('(\[.*\])', "").astype(float) * 1_000_000  

# Removes things in brackets and parenthesis
ww1_df['Total military deaths from all causes'] = ww1_df['Total military deaths from all causes'].str.replace('(\[..\])', "").str.replace('(\(.*\))', "")
# Averages the values
ww1_df['Total military deaths from all causes'] = ww1_df['Total military deaths from all causes'].str.replace(',', "").str.split("to", expand=True).astype(float).mean(axis=1).round()

# Removes things in brackets and parenthesis
ww1_df['Civilian deaths due to indirect military action'] = ww1_df['Civilian deaths due to indirect military action'].str.replace('(\[..\])', "").str.replace('(\(.*\))', "")
# Averages the values
ww1_df['Civilian deaths due to indirect military action'] = ww1_df['Civilian deaths due to indirect military action'].str.replace(',', "").str.split("to", expand=True).astype(float).mean(axis=1).round()


# Removes things in brackets and parenthesis
ww1_df['Civilian deaths due to direct military action'] = ww1_df['Civilian deaths due to direct military action'].str.replace('(\[..\])', "").str.replace('(\(.*\))', "")
# Averages the values
ww1_df['Civilian deaths due to direct military action'] = ww1_df['Civilian deaths due to direct military action'].str.replace(',', "").str.split("to", expand=True).astype(float).mean(axis=1).round()




C:\Users\zayne\AppData\Local\Temp\ipykernel_11632\926872877.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  ww1_df['Population'] = ww1_df['Population'].str.replace('(\[.*\])', "").astype(float) * 1_000_000
C:\Users\zayne\AppData\Local\Temp\ipykernel_11632\926872877.py:36: FutureWarning: The default value of regex will change from True to False in a future version.
  ww1_df['Total military deaths from all causes'] = ww1_df['Total military deaths from all causes'].str.replace('(\[..\])', "").str.replace('(\(.*\))', "")
C:\Users\zayne\AppData\Local\Temp\ipykernel_11632\926872877.py:41: FutureWarning: The default value of regex will change from True to False in a future version.
  ww1_df['Civilian deaths due to indirect military action'] = ww1_df['Civilian deaths due to indirect military action'].str.replace('(\[..\])', "").str.replace('(\(.*\))', "")
C:\Users\zayne\AppData\Local\Temp\ipykernel_11632\926872877.py:47: FutureWarning: The

In [100]:
ww1_df

,Nation,Population,Total military deaths from all causes,Civilian deaths due to direct military action,Civilian deaths due to indirect military action,Total deaths,Deaths as percent of population,Military wounded
0,Australia b,5000000.0,60740.0,NaN,NaN,"59,330to 62,149",1.2% to 1.2%,"152,171[21]"
1,Canada d,7200000.0,60818.0,1963.0,NaN,"58,639 to 66,996",0.8% to 0.9%,"149,732[21]"
2,India g,315100000.0,69177.0,NaN,NaN,"64,449 to 73,905",0% to 0%,"69,214[21]"
3,New Zealand l,1100000.0,17386.0,NaN,NaN,"16,711 to 18,060",1.5% to 1.6%,"41,317[21]"
4,Newfoundland m,200000.0,1387.0,NaN,NaN,"1,204to 1,570",0.6% to 0.8%,"2,314[21]"
5,South Africa r,6000000.0,8424.0,NaN,NaN,"7,121 to 9,726",0.1% to 0.2%,"12,029[21]"
6,United Kingdom(and Colonies) s,45400000.0,887858.0,16829.0,107000.0,"867,829 to 1,011,687",1.9% to 2.2%,"1,675,000s1"
7,Sub-total for British Empire,380000000.0,1033859.0,18829.0,107000.0,"1,077,283to 1,244,093",0.3% to 0.3%,2101077
8,Belgium c,7400000.0,48404.0,23700.0,62000.0,"123,870to 144,337",1.7% to 2%,"44,686[33]"
9,France e,39600000.0,1377400.0,40000.0,300000.0,"1,697,000to 1,737,800",4.3% to 4.4%,"4,266,000[33]"


C:\Users\zayne\AppData\Local\Temp\ipykernel_11632\192642330.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  ww1_df['Civilian deaths due to indirect military action'] = ww1_df['Civilian deaths due to indirect military action'].str.replace('(\[..\])', "").str.replace('(\(.*\))', "")


In [10]:
#Clean for merge
names = ["Nation", "Population", "Total military deaths from all causes", "Civilian deaths due to direct military action", 
"Civilian deaths due to indirect military action","Total deaths", 'Deaths as percent of population', 'Military wounded']
ww1_df.columns = names
ww2_df.columns = names

ww1_df['Population'] = int(ww1_df['Population']) * 1_000_000
ww2_df["War"] = "World War 2"
ww1_df["War"] = "World War 1"


TypeError: cannot convert the series to <class 'int'>

In [6]:
#Concat the Data
data = pd.concat([ww1_df, ww2_df])

# data.to_csv("Uncleaned_data.csv")



In [7]:
data

,Nation,Population,Total military deaths from all causes,Civilian deaths due to direct military action,Civilian deaths due to indirect military action,Total deaths,Deaths as percent of population,Military wounded,War
0,Australia b,5.0 Million,"59,330[20]to 62,149[11]",NaN,NaN,"59,330to 62,149",1.2% to 1.2%,"152,171[21]",World War 1
1,Canada d,7.2 Million,"56,639[21] to 64,996[11]","1,963[23][24]",NaN,"58,639 to 66,996",0.8% to 0.9%,"149,732[21]",World War 1
2,India g,315.1 Million,"64,449[21] to 73,905[11]",NaN,NaN,"64,449 to 73,905",0% to 0%,"69,214[21]",World War 1
3,New Zealand l,1.1 Million,"16,711[21] to 18,060[11]",NaN,NaN,"16,711 to 18,060",1.5% to 1.6%,"41,317[21]",World War 1
4,Newfoundland m,0.2 Million,"1,204[21] to 1,570[26](included with UK)",NaN,NaN,"1,204to 1,570",0.6% to 0.8%,"2,314[21]",World War 1
...,...,...,...,...,...,...,...,...,...
58,United KingdomBE including Crown Colonies,"47,760,000[141]","383,700[142]","67,200[143][144]",NaN,450900,0.94,"376,239[20]",World War 2
59,United StatesBF,"131,028,000[16]","407,300BF1","12,100BF2",NaN,419400,0.32,"671,801[20]",World War 2
60,YugoslaviaBG,"15,490,000[145]","300,000[146] to 446,000[147]","581,000[147] to 1,400,000[146]",NaN,"1,027,000[147] to 1,700,000[146]",6.63 to 10.97,"425,000[20]",World War 2
61,Other nationsBH,300000000,NaN,NaN,NaN,NaN,NaN,NaN,World War 2
